In [2]:
import librosa
import os
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import csv
import torch.optim as optim
#import feature_extraction.py
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class Network(nn.Module):

    def __init__(self): # You can add any extra arguments as you wish

        super(Network, self).__init__()
        
        self.lstm = nn.LSTM(input_size=431, hidden_size = 256, num_layers=2, dropout=.1, bidirectional=True, batch_first=True)# TODO: # Create a single layer, uni-directional LSTM with hidden_size = 256

        self.classification = nn.Sequential(nn.ReLU(),
                                            nn.Linear(512,1))
            # TODO: Create a single classification layer using nn.Linear()

    def forward(self, x, x_length): # TODO: You need to pass atleast 1 more parameter apart from self and x
        x = torch.transpose(x, 1, 2)
        packed_input = pack_padded_sequence(x,x_length, batch_first = True, enforce_sorted=False) # TODO: Pack the input with pack_padded_sequence. Look at the parameters it requires

        
        #packed_input = pack_padded_sequence(x,length.clamp(max=max_expected_len), batch_first = True, enforce_sorted=False)
        out1, (out2, out3) =  self.lstm(packed_input) # TODO: Pass packed input to self.lstm
        # As you may see from the LSTM docs, LSTM returns 3 vectors. Which one do you need to pass to the next function?
        out, lengths  = pad_packed_sequence(out1, batch_first=True) # TODO: Need to 'unpack' the LSTM output using pad_packed_sequence
        #pdb.set_trace()
        out = self.classification(out) # TODO: Pass unpacked LSTM output to the classification layer

        #Is dimension 2 or 3? Or something else?
   
        #m = n.LogSoftmax(dim=2) # Optional: Do log softmax on the output. Which dimension?
        out = F.log_softmax(out,dim=2)


        return out, lengths # TODO: Need to return 2 variables

model = model = Network().to(device)